In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
import math
from tqdm import tqdm
from Data_split import data_split, pd_ret_split, R2_score
# need some extra function like data_split

# Perform PLS to full data

In [ ]:

dt = pd.read_csv('merge_data.csv')
ncom_list = [ ]
R2_list = [ ]
dt


In [ ]:
for i in tqdm(range(199001,202001,100)):
    
    train,val,test = data_split(i, dt)
    x_train, y_train = pd_ret_split(train)
    x_val, y_val = pd_ret_split(val)
    y_val = np.array(y_val)
    y_val = y_val.reshape(len(y_val),1)
    x_test, y_test = pd_ret_split(test)
    y_test = np.array(y_test)
    y_test = y_test.reshape(len(y_test),1)
    
    best_valR2 = float("-Inf")
    best_pcr = None
    
                         
    for K in range(5,50,10):
        pls = PLSRegression(n_components = K)
        pls_norm = make_pipeline(StandardScaler(), pls)
        pls_norm.fit(x_train,y_train)
    
        val_pred = pls_norm.predict(x_val)
        cval_R2 = R2_score(y_val, val_pred) #annual R2 score
            
        if(cval_R2 > best_valR2):
            best_valR2 = cval_R2
            best_pls_norm = pls_norm
            best_ncom = K

        
    test_pred = best_pls_norm.predict(x_test)
    testR2 = R2_score(y_test, test_pred)

    ncom_list.append(best_ncom)
    R2_list.append(testR2)

In [4]:
ncom_list = pd.DataFrame(ncom_list) 
R2_list = pd.DataFrame(R2_list)

In [ ]:
ncom_list, R2_list

In [ ]:
R2_PLS=R2_list.mean(axis=0)




In [ ]:
best_year=np.argmax(R2_list)
best_k = ncom_list.iloc[best_year,0]
best_year= 199001+best_year*100
best_R2 = np.max(R2_list)
best_year,best_k, best_R2

# Analyze variable importance of PLS

    From above, we attain the testing year with best R2 and best component number. Based on this PLS model, we measure each predictors's importance using the R2 reduction from setting all values of this specified predictor to zero, while hoilding the remaining model estimates fiexed.

In [8]:
var = dt.columns

In [9]:
var = var.drop(['permno','yyyymm','excess_ret'])
var

In [ ]:
dt_pd = dt.copy()
dt_pd1 = dt_pd.drop(var[0],axis = 1)
dt_pd.var[0] = 0


In [ ]:
var_result = list()
sum = 0
for j in tqdm(var):
    dt_pd = dt.copy()
    dt_pd[j]=0

        
         
    train,val,test = data_split(best_year, dt_pd)
    x_train, y_train = pd_ret_split(train)
    x_test, y_test = pd_ret_split(test)
    y_test = np.array(y_test)
    y_test = y_test.reshape(len(y_test),1)

    pls = PLSRegression(n_components = best_k)
    pls_norm = make_pipeline(StandardScaler(), pls)
    pls_norm.fit(x_train,y_train)
    
    test_pred = pls_norm.predict(x_test)
    testR2 = R2_score(y_test, test_pred)
    Im = abs(best_R2 - testR2)
    Im = Im.to_numpy()
    sum = sum +Im
    var_result.append([j, Im])
   



In [ ]:
for i in range(0,len(var_result)):
    var_result[i][1] = var_result[i][1]/sum

var_result = pd.DataFrame(var_result,columns = ['Variable','Importance'])
var_result.sort_values(by = 'Importance', ascending = False)


In [ ]:
var_result.head(20)